<a href="https://colab.research.google.com/github/jasleenkaursandhu/Reproducing-chest-xray-report-generation-boag/blob/main/extract_densenet_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Feature Extraction for k-NN Chest X-ray Report Generation
# Based on the Boag et al. paper "Baselines for Chest X-Ray Report Generation"
# This notebook extracts DenseNet121 features from chest X-ray images

import numpy as np
import pandas as pd
import os
import pydicom
import tensorflow as tf
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from PIL import Image
import tqdm
import pickle
import gc
from sklearn.metrics.pairwise import cosine_similarity

# Temporarily disable certificate verification (Not recommended for production)
import ssl
os.environ['PYTHONHTTPSVERIFY'] = '0'
ssl._create_default_https_context = ssl._create_unverified_context

# Define paths
base_path = '/Users/simeon/Documents/DLH/content/mimic-cxr-project'
data_dir = os.path.join(base_path, 'data')
files_path = os.path.join(base_path, 'new_files')
output_dir = os.path.join(base_path, 'output')
features_dir = os.path.join(base_path, 'features')

# Create output directories if they don't exist
os.makedirs(output_dir, exist_ok=True)
os.makedirs(features_dir, exist_ok=True)

# Load train and test data
train_df = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
test_df = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')

print(f"Train data shape: {train_df.shape}")
print(f"Test data shape: {test_df.shape}")

# Load pre-trained DenseNet121 model
# Per the paper: "We use a DenseNet121 to extract features of size 8 × 8 × 1024,
# which are then globally mean-pooled to a final, 1024-dimensional representation"
model = DenseNet121(weights='imagenet', include_top=False, pooling='avg')
print("Loaded DenseNet121 model with global average pooling")

# Function to extract features from a DICOM image
def extract_features(dicom_path):
    try:
        # Read the DICOM file
        ds = pydicom.dcmread(dicom_path)

        # Convert to image format
        pixel_array = ds.pixel_array

        # Normalize pixel values
        pixel_array = pixel_array / np.max(pixel_array)

        # Convert to uint8
        img = np.uint8(pixel_array * 255)

        # Convert to RGB (DenseNet expects 3 channels)
        if len(img.shape) == 2:
            # Grayscale to RGB
            img_rgb = np.stack([img, img, img], axis=2)
        elif img.shape[2] == 1:
            # Single channel to RGB
            img_rgb = np.concatenate([img, img, img], axis=2)
        else:
            img_rgb = img

        # Resize to 224x224 (expected by DenseNet)
        pil_img = Image.fromarray(img_rgb)
        pil_img = pil_img.resize((224, 224))

        # Convert to numpy array and preprocess
        img_array = np.array(pil_img)
        img_array = preprocess_input(img_array)

        # Add batch dimension
        img_array = np.expand_dims(img_array, axis=0)

        # Extract features
        features = model.predict(img_array, verbose=0)

        return features.flatten()
    except Exception as e:
        print(f"Error processing {dicom_path}: {e}")
        return None

# Process train images in batches to avoid memory issues
batch_size = 50
densenet_vecs = {}

# Process train images
for idx, row in tqdm.tqdm(train_df.iterrows(), total=len(train_df), desc="Processing train images"):
    if idx % batch_size == 0:
        print(f"Processed {idx}/{len(train_df)} train images")

    dicom_id = row['dicom_id']
    subject_id = row['subject_id']
    study_id = row['study_id']

    # Construct path to the DICOM file
    subject_prefix = f"p{str(subject_id)[:2]}"
    subject_dir = f"p{subject_id}"
    study_dir = f"s{study_id}"
    dicom_file = f"{dicom_id}.dcm"
    dicom_path = os.path.join(files_path, subject_prefix, subject_dir, study_dir, dicom_file)

    if os.path.exists(dicom_path):
        features = extract_features(dicom_path)
        if features is not None:
            densenet_vecs[dicom_id] = features

    # Save intermediate results
    if (idx + 1) % 500 == 0 or idx == len(train_df) - 1:
        print(f"Saving intermediate results: {len(densenet_vecs)} vectors")
        with open(os.path.join(features_dir, 'densenet121_train.pkl'), 'wb') as f:
            pickle.dump(densenet_vecs, f)

    # Clean up memory
    if idx % 100 == 0:
        gc.collect()

# Save the final vectors
with open(os.path.join(features_dir, 'densenet121_train.pkl'), 'wb') as f:
    pickle.dump(densenet_vecs, f)

# Also save as numpy arrays for easier processing
train_dicom_ids = list(densenet_vecs.keys())
train_features = np.array([densenet_vecs[dicom_id] for dicom_id in train_dicom_ids])

np.save(os.path.join(features_dir, 'train_dicom_ids.npy'), np.array(train_dicom_ids))
np.save(os.path.join(features_dir, 'train_densenet_features.npy'), train_features)

print(f"Saved DenseNet features for {len(densenet_vecs)} train images")

# Process test images
test_densenet_vecs = {}

for idx, row in tqdm.tqdm(test_df.iterrows(), total=len(test_df), desc="Processing test images"):
    if idx % batch_size == 0:
        print(f"Processed {idx}/{len(test_df)} test images")

    dicom_id = row['dicom_id']
    subject_id = row['subject_id']
    study_id = row['study_id']

    # Construct path to the DICOM file
    subject_prefix = f"p{str(subject_id)[:2]}"
    subject_dir = f"p{subject_id}"
    study_dir = f"s{study_id}"
    dicom_file = f"{dicom_id}.dcm"
    dicom_path = os.path.join(files_path, subject_prefix, subject_dir, study_dir, dicom_file)

    if os.path.exists(dicom_path):
        features = extract_features(dicom_path)
        if features is not None:
            test_densenet_vecs[dicom_id] = features

    # Save intermediate results
    if (idx + 1) % 100 == 0 or idx == len(test_df) - 1:
        print(f"Saving intermediate results: {len(test_densenet_vecs)} vectors")
        with open(os.path.join(features_dir, 'densenet121_test.pkl'), 'wb') as f:
            pickle.dump(test_densenet_vecs, f)

    # Clean up memory
    if idx % 50 == 0:
        gc.collect()

# Save the test vectors
with open(os.path.join(features_dir, 'densenet121_test.pkl'), 'wb') as f:
    pickle.dump(test_densenet_vecs, f)

# Also save as numpy arrays for easier processing
test_dicom_ids = list(test_densenet_vecs.keys())
test_features = np.array([test_densenet_vecs[dicom_id] for dicom_id in test_dicom_ids])

np.save(os.path.join(features_dir, 'test_dicom_ids.npy'), np.array(test_dicom_ids))
np.save(os.path.join(features_dir, 'test_densenet_features.npy'), test_features)

print(f"Saved DenseNet features for {len(test_densenet_vecs)} test images")

# Modified section: Find k-NN for multiple values of k (1, 50, 100, 200)
print("Finding k-NN (nearest neighbors) for each test image with k=1, 50, 100, 200...")

# Convert feature dictionaries to arrays for faster processing
train_dicom_ids = list(densenet_vecs.keys())
train_features = np.array([densenet_vecs[dicom_id] for dicom_id in train_dicom_ids])

# Define the k values we want to compute
k_values = [1, 10, 50, 100, 200]
max_k = max(k_values)  # We'll compute up to the maximum k and then subset

# Find k-NN for each test image and for each k
knn_neighbors = {k: {} for k in k_values}

for test_dicom, test_feature in tqdm.tqdm(test_densenet_vecs.items(), desc="Finding k-NN"):
    # Reshape test features for cosine similarity
    test_feature_reshaped = test_feature.reshape(1, -1)

    # Compute similarity to all training images
    similarities = cosine_similarity(test_feature_reshaped, train_features)[0]

    # Get indices of the most similar images (for the maximum k)
    top_indices = similarities.argsort()[-max_k:][::-1]

    # Get the corresponding dicom IDs
    nearest_neighbors = [train_dicom_ids[idx] for idx in top_indices]

    # Store in our neighbors dictionary for each k value
    for k in k_values:
        knn_neighbors[k][test_dicom] = nearest_neighbors[:k]

# Save the k-NN neighbors for each k
for k in k_values:
    neighbors_path = os.path.join(output_dir, f'{k}nn_neighbors.pkl')
    with open(neighbors_path, 'wb') as f:
        pickle.dump(knn_neighbors[k], f)
    print(f"Saved {k}-NN neighbors for {len(knn_neighbors[k])} test images")

Train data shape: (4291, 3)
Test data shape: (1757, 3)


2025-04-25 13:18:53.960058: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-04-25 13:18:53.960242: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-04-25 13:18:53.960249: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
2025-04-25 13:18:53.960281: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-04-25 13:18:53.960291: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Loaded DenseNet121 model with global average pooling


Processing train images:   0%|          | 0/4291 [00:00<?, ?it/s]

Processed 0/4291 train images


...

Processed 1750/1757 test images


Processing test images: 100%|██████████| 1757/1757 [03:26<00:00,  8.52it/s]


Saving intermediate results: 1757 vectors
Saved DenseNet features for 1757 test images
Finding k-NN (nearest neighbors) for each test image with k=1, 50, 100, 200...


Finding k-NN: 100%|██████████| 1757/1757 [00:19<00:00, 88.57it/s] 


Saved 1-NN neighbors for 1757 test images
Saved 10-NN neighbors for 1757 test images
Saved 50-NN neighbors for 1757 test images
Saved 100-NN neighbors for 1757 test images
Saved 200-NN neighbors for 1757 test images
